In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta
import re
import warnings
import math

In [2]:
warnings.simplefilter('ignore')

## Flightplan

In [ ]:
arr_df = pd.DataFrame()
dep_df = pd.DataFrame()

In [ ]:
daterange = pd.date_range(start='12/22/2019', end='12/28/2019').strftime('%Y%m%d')

In [ ]:
for date in daterange.tolist():
    arr = pd.read_excel(f'../data/flightplan/arr_{date}.xlsx', header = 0)
    dep = pd.read_excel(f'../data/flightplan/dep_{date}.xlsx', header = 0)
    datee = datetime.strptime(date,'%Y%m%d').strftime('%Y-%m-%d')
    arr['SDT'] = datee
    dep['SDT'] = datee
    arr_df = pd.concat([arr_df, arr])
    dep_df = pd.concat([dep_df, dep])

In [ ]:
arr_df = arr_df.drop('Seq', axis = 1)
dep_df = dep_df.drop('Seq', axis = 1)

In [ ]:
arr_df[arr_df['FLT'] == 'UAL893']

In [ ]:
arr_df.to_csv('../input/arr_fp_20191222-20191228.csv')
dep_df.to_csv('../input/dep_fp_20191222-20191228.csv')

In [ ]:
# ACDM하고 데이터를 합칠 수 있을까?? -> 활주로 정보 넣기
# Traj0501 = io.loadmat(data_file)


## Takeoff Distance

T/O Distance 

- OAT, Pressure Altitude <br>
	-> METAR에서 가져올 수 있을 듯
	-> Density Altitude를 구하고 변수로 넣기

- Gross Weight <br>
	-> 이게 애매하단 말이지...
	-> 기종 별로는 할 수 있음

- Wind components <br>
	-> METAR에서 가져오기
	-> Headwind, Tailwind 성분만 가지고 하기

- RWY condition <br>
	-> METAR에서 가져오기
	-> Wet rwy 
	-> RWY slope는 rksi = 0


*** T/O speed varies with T/O Weight

### METAR

In [3]:
metar_data = pd.read_excel('../data/METAR_20191222-20191228.xlsx', header = 0)
metar_data = metar_data.iloc[::-1].reset_index(drop=True).drop('Seq', axis = 1)

In [4]:
# time

metar_data['IssueTime'] = np.zeros(len(metar_data))
pattern_time = '[0-9][0-9][0-9][0-9][0-9][0-9]Z'

for i in range(len(metar_data)):
    time = re.search(pattern_time, metar_data['Original Message'][i])
    time = time.group()[:-1]
    time = datetime.strptime('201912'+time, '%Y%m%d%H%M') + timedelta(hours=9)
    metar_data['IssueTime'][i] = time

In [5]:
# pressure column만들기

metar_data['Pressure'] = np.zeros(len(metar_data))
pattern_pres = 'Q[0-9][0-9][0-9][0-9]'

for i in range(len(metar_data)):
    pres = re.search(pattern_pres, metar_data['Original Message'][i])
    metar_data['Pressure'][i] = int(pres.group()[1:])

In [6]:
# dewpoint column 만들기

metar_data['DuePoint'] = np.zeros(len(metar_data))
pattern_due = '/..[0-9]|/.[0-9]'

for i in range(len(metar_data)):
    due = re.search(pattern_due, metar_data['Original Message'][i])
    if due.group()[1] == 'M':
        due = int(due.group()[2:]) * (-1)
    else:
        due = int(due.group()[1:])
        
    metar_data['DuePoint'][i] = due

In [7]:
# density altitude 계산하기

metar_data['DensityAltitude'] = np.zeros(len(metar_data))
fe = 7    # field elevation in meters

for i in range(len(metar_data)):
    pres = metar_data['Pressure'][i]
    temp = metar_data['Temp'][i]
    due = metar_data['DuePoint'][i]

    # station pressure (https://www.weather.gov/media/epz/wxcalc/stationPressure.pdf)
    station_pressure_inHg = pres * 0.0295300  * (((288 - (0.0065 * fe))/288)**(5.2561))
    station_pressure_mil = station_pressure_inHg * 33.8639

    # density altitude (https://www.weather.gov/media/epz/wxcalc/densityAltitude.pdf)
    vapor_pressure = 6.11 * (10 **((7.5 * due) / (237.7 + due)))
    tv_kelvin = (temp + 273.15) / (1 - ((vapor_pressure / station_pressure_mil) * (1 - 0.622)))  # Kelvin
    tv_rankine = (tv_kelvin * 9 / 5)
    da = 145366 * ( 1 - ((17.326 * station_pressure_mil * 0.029530 / tv_rankine )**(0.235)))
    
    metar_data['DensityAltitude'][i] = da


In [8]:
# Headwind Components - 아마 바람 방향 바뀌면 바로 활주로 방향 바꾸니까 그냥 cos 때리자

# rksi rwy15,16 true bearing = 144.66 degree

metar_data['Headwind'] = np.zeros(len(metar_data))
metar_data['Crosswind'] = np.zeros(len(metar_data))

pattern_wind = '[0-9]...[0-9]KT'

for i in range(len(metar_data)):
    wind = re.search(pattern_wind, metar_data['Original Message'][i])
    metar_data['WD'][i] = int(wind.group()[0:3])
    deg = metar_data['WD'][i] - 144.66
    headwind = metar_data['WS'][i] * np.cos(deg * math.pi/180)
    crosswind = metar_data['WS'][i] * np.sin(deg * math.pi/180)
    metar_data['Headwind'][i] = abs(headwind)
    metar_data['Crosswind'][i] = abs(crosswind)

In [9]:
# Wet Runway

# 비, 눈 등 있으면 wet으로 1로 넣기 

metar_data['Wet'] = np.zeros(len(metar_data))
pattern_wc = 'TS|DZ|RA|SN|SG|IC|PL|GR|GS'

for i in range(len(metar_data)):
    try:
        wc = re.search(pattern_wc, metar_data['Original Message'][i])
        wc.group()
        wc = 1
    except:
        wc = 0    
        
    metar_data['Wet'][i] = wc

In [16]:
metar_data

,Type,Port,STS,VIS,WD,WS,Temp,Original Message,IssueTime,Pressure,DuePoint,DensityAltitude,Headwind,Crosswind,Wet
0,METAR,RKSI,7,5000,130,6,3,211500Z 13006KT 5000 BR FEW015 SCT040 BKN200 0...,2019-12-22 00:00:00,1024.0,0.0,-1701.232595,5.804668,1.518496,0.0
1,METAR,RKSI,7,5000,140,6,2,211530Z 14006KT 5000 BR FEW015 SCT040 BKN200 0...,2019-12-22 00:30:00,1024.0,0.0,-1826.665476,5.980166,0.487456,0.0
2,METAR,RKSI,7,5000,130,6,3,211600Z 13006KT 5000 BR FEW015 SCT040 BKN200 0...,2019-12-22 01:00:00,1024.0,0.0,-1701.232595,5.804668,1.518496,0.0
3,METAR,RKSI,0,6000,90,5,3,211630Z 09005KT 040V130 6000 FEW015 SCT040 BKN...,2019-12-22 01:30:00,1024.0,0.0,-1701.232595,2.892136,4.078670,0.0
4,METAR,RKSI,0,6000,70,4,2,211700Z 07004KT 6000 SCT040 BKN200 02/M00 Q102...,2019-12-22 02:00:00,1024.0,0.0,-1826.665476,1.058185,3.857492,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,METAR,RKSI,0,9000,160,11,5,281230Z 16011KT 9000 NSC 05/M01 Q1028 NOSIG=,2019-12-28 21:30:00,1028.0,-1.0,-1592.394508,10.608103,2.910010,0.0
332,METAR,RKSI,0,8000,150,9,4,281300Z 15009KT 8000 NSC 04/M01 Q1029 NOSIG=,2019-12-28 22:00:00,1029.0,-1.0,-1750.512210,8.960940,0.837591,0.0
333,METAR,RKSI,0,9000,150,7,4,281330Z 15007KT 9000 NSC 04/M02 Q1029 NOSIG=,2019-12-28 22:30:00,1029.0,-2.0,-1755.644056,6.969620,0.651460,0.0
334,METAR,RKSI,0,10000,120,6,4,281400Z 12006KT 090V160 CAVOK 04/M03 Q1028 NOSIG=,2019-12-28 23:00:00,1028.0,-3.0,-1726.776994,5.452798,2.503396,0.0


In [11]:
metar_data.to_csv('../input/metar_20191222-20191228.csv')

In [ ]:
# DA in feets

"""
pres = 1016.2552
temp = 24
due = 23
fe = 17

# station pressure (https://www.weather.gov/media/epz/wxcalc/stationPressure.pdf)
station_pressure_inHg = pres * 0.0295300  * (((288 - (0.0065 * fe))/288)**(5.2561))
station_pressure_mil = station_pressure_inHg * 33.8639

# density altitude (https://www.weather.gov/media/epz/wxcalc/densityAltitude.pdf)
vapor_pressure = 6.11 * (10 **((7.5 * due) / (237.7 + due)))
tv_kelvin = (temp + 273.15) / (1 - ((vapor_pressure / station_pressure_mil) * (1 - 0.622)))  # Kelvin
tv_rankine = (tv_kelvin * 9 / 5)
da = 145366 * ( 1 - ((17.326 * station_pressure_mil * 0.029530 / tv_rankine )**(0.235)))
"""